In [1]:
from textattack.augmentation import \
    EasyDataAugmenter, BackTranslationAugmenter, WordNetAugmenter, CLAREAugmenter, \
    CheckListAugmenter, EmbeddingAugmenter, DeletionAugmenter, CharSwapAugmenter

from itertools import chain
import warnings
warnings.filterwarnings("ignore")

/home/student/.envs/textattack/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-17 15:42:55.264800: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-17 15:42:55.264816: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import pandas as pd
df = pd.read_csv('../data/train/cr/train.txt', sep='|', header=None, names=['text'])
df['class'] = df['text'].apply(lambda x: x.split('\t')[0])
df['text'] = df['text'].apply(lambda x: x.split('\t')[1])
df = df[['class', 'text']]

df_test = df[0:200]
df_test

,class,text
0,0,The pluger gets stuck & you have to manually e...
1,1,"The Hitachi is made in Malasyia, and looked ch..."
2,0,i must have heard this about a dozen times ove...
3,1,This model appears to be especially good.
4,0,"in any case , navigation by artist / album is ..."
...,...,...
195,1,For the cost of this product i could not ask ...
196,1,After comparing the included Creative earbuds ...
197,1,Better for the Zen Micro is Outlook compatibil...
198,1,"I just set it up manually , which is very easy ."


In [3]:
augmenter_dict = { 
    'eda_augmenter':EasyDataAugmenter(pct_words_to_swap=0.2,transformations_per_example=4),
    'wordnet_augmenter':WordNetAugmenter(pct_words_to_swap=0.2,transformations_per_example=4),
    'backtranslation_augmenter':BackTranslationAugmenter(pct_words_to_swap=0.2,transformations_per_example=4),
    'embedding_augmenter':EmbeddingAugmenter(pct_words_to_swap=0.2,transformations_per_example=4),
    'deletion_augmenter':DeletionAugmenter(pct_words_to_swap=0.2,transformations_per_example=4),
    'charswap_augmenter':CharSwapAugmenter(pct_words_to_swap=0.2,transformations_per_example=4)
}

[nltk_data] Downloading package omw-1.4 to /home/student/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/student/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
augmenter_list = ['eda','wordnet','backtranslation','embedding','deletion','charswap']

In [5]:
def augment_text(df,aug_method,fraction=0.5,label_column='class',target_column='text',pct_words_to_swap=0.2,transformations_per_example=4,include_original=True):
    augmenter = augmenter_dict[aug_method.lower()+'_augmenter']
    augmenter.pct_words_to_swap = pct_words_to_swap
    augmenter.transformations_per_example = transformations_per_example
    # print('Augmenting with',str(augmenter))
    # print('percentage of words to swap:',augmenter.pct_words_to_swap)
    # print('number of transformations per example:',augmenter.transformations_per_example)
    df = df.sample(frac=fraction)
    text_list , class_list = [], []
    for c, txt in zip(df[label_column], df[target_column]):

        res = augmenter.augment(txt)
        if include_original:
            text_list.append(txt)
            class_list.append(c)
            for i in res:
                text_list.append(i)
                class_list.append(c)
        else:
            for i in range(len(res)):
                text_list.append(res[i])
                class_list.append(c)

    df_augmented = pd.DataFrame({target_column: text_list, label_column: class_list})

    return df_augmented

In [6]:
mydf = augment_text(df_test,'eda',fraction=1,label_column='class',target_column='text',pct_words_to_swap=0.2,transformations_per_example=4,include_original=False)

In [7]:
mydf

,text,class
0,"considering the price and specs , this might b...",1
1,"considering the price and specs , this might b...",1
2,"considering the Price and specs , this might b...",1
3,"the price and specs , this might be the best b...",1
4,And it fits right in my much easier than my old.,1
...,...,...
795,screw i love it ! !,1
796,The problem is the installation.,0
797,The biggest problem is trouble the installation.,0
798,is biggest problem The the installation.,0
